### 读取背景音乐

In [1]:
import wave
import numpy as np

# music = wave.open(r'background.wav', mode='rb')
# music.getparams()
# bg_data = music.readframes(nframes=32191488)
# bg_data = np.frombuffer(bg_data, dtype=np.int16)#音频的格式通常都是16位；buffer缓冲器
# bg_data.shape

# bg_data = np.zeros([44100*15*60], dtype=np.int16)

In [2]:
ding = wave.open(r'ding.wav', mode='rb')
ding.getparams()

_wave_params(nchannels=1, sampwidth=2, framerate=44100, nframes=39710, comptype='NONE', compname='not compressed')

In [3]:
nframes = 39710
nchannels=1
ding_data = ding.readframes(nframes=nframes)
ding_data = np.frombuffer(ding_data, dtype=np.int16)
# ding_data = ding_data.reshape(nframes, 2)
ding_data.shape

(39710,)

### 读取视频息肉出现时间点

In [4]:
import sys
import os
import subprocess
sys.path.append(r'../../NLP/MedicalRecord/Lib')

from MRRecordUtil import load_sheet_arr_dict

In [10]:
# 计算视频秒数
video_fps = 18
def calc_video_len(video_name, base_dir):
    img_folder = os.path.join(os.path.join(base_dir, 'images'), video_name)
    return len(os.listdir(img_folder)) // video_fps

# 合成视频音频
def gen_audio(ding_data, video_secs, framerate, polp_time_arr):
    print(video_secs, framerate, video_secs*framerate)
    print(polp_time_arr)
    bg_data = np.zeros([video_secs*framerate], dtype=np.int16)
    bg_len = bg_data.shape[0]
    start_frames = []
    if ding_data is not None:
        ding_len = ding_data.shape[0]
#         ding_sec = ding_len / framerate
#         print(ding_len)
        print('bg_len: %s, ding_len: %s' % (bg_len, ding_len))
        last_frame = -1
        for ptime in polp_time_arr:
            p_frame = int(ptime * framerate)
            if (last_frame == -1 or p_frame - last_frame > ding_len) and p_frame < bg_len:
                start_frame = p_frame
                end_frame = start_frame + ding_len
                end_frame = end_frame if bg_len > end_frame else bg_len
#                 print(ptime, start_frame, end_frame)
#                 bg_data[start_frame:end_frame, :] = ding_data[0:(end_frame-start_frame), :]
                bg_data[start_frame:end_frame] = ding_data[0:(end_frame-start_frame)]
                last_frame = p_frame
                start_frames.append(start_frame)
            else:
                print(ptime, p_frame - last_frame)
    print(start_frames)
    print(len(polp_time_arr), len(start_frames))
    print([f / framerate for f in start_frames])
    return bg_data

# 写音频
def write_audio(audio_data, secs, framerate=44100, nchannels=1, sampwidth=2):
    if os.path.exists('./generated.wav'):
        os.remove('./generated.wav')
    fp = wave.Wave_write('./generated.wav')
    #设置fp的参数
    fp.setframerate(framerate)
    fp.setnframes(secs*framerate)
    fp.setnchannels(nchannels)
    fp.setsampwidth(sampwidth)

    #写入目录
    fp.writeframes(audio_data.tobytes())
    fp.close()
    
# 读取excel记录视频数据
def get_video_data_from_sheet(sheet_data):
    video_data = []
    for r in sheet_data:
        r_data = {}
        r_data['视频名'] = '%d' % r['视频名']
        r_data['所选参数'] = r['所选参数']
#         r_data['假阳性出现时间'] = [float(e) for e in r['假阳性出现时间'].split('\n') if e != '']
        r_data['真阳性出现时间'] = [float(e.split('-')[0]) / 18 for e in r['真阳性出现帧'].split('\n') if e != '']
        r_data['假阳性出现时间'] = [float(e.split(' ')[0]) for e in r['假阳性出现时间_开始帧_结束帧'].split('\n') if e != '']
        r_data['息肉出现时间点'] = sorted(r_data['假阳性出现时间'] + r_data['真阳性出现时间'])
        video_data.append(r_data)
    return video_data

def add_audio_for_video_data(v_data, video_folder, base_dir):
    video_secs = calc_video_len(v_data['视频名'], base_dir)
    print('video seconds: %s' % video_secs)
    audo_data = gen_audio(ding_data, video_secs=video_secs, framerate=44100, polp_time_arr=v_data['息肉出现时间点'])
    write_audio(audo_data, video_secs, framerate=44100)
    
    video_path = os.path.join(video_folder, '%s_%s.avi' % (v_data['视频名'], v_data['所选参数']))
    trans_video_path = os.path.join(video_folder, '%s_%s_audio.mp4' % (v_data['视频名'], v_data['所选参数']))
    cmd = 'ffmpeg -i generated.wav -i %s %s' % (video_path, trans_video_path)
    subprocess.call(cmd)

In [11]:
excel = '跑视频信息表.xls'
for base_dir in [r'D:\项目资料\息肉假阳性\20221103']:
    video_folder = os.path.join(base_dir, 'videos_gen')
    sheet_data = load_sheet_arr_dict(os.path.join(base_dir, excel), 'Sheet1')
    video_data = get_video_data_from_sheet(sheet_data)
    
    for v_data in video_data:
        if len(v_data['视频名']) > 0:# and v_data['视频名'] in ['1307'] and v_data['所选参数'] == 0.02:
            print('processing ', v_data['视频名'], v_data['所选参数'])
            add_audio_for_video_data(v_data, video_folder, base_dir)


processing  3419 0.2
video seconds: 481
481 44100 21212100
[60.889, 62.5, 63.278, 111.5, 127.778, 300.333, 456.056, 465.167, 475.333]
bg_len: 21212100, ding_len: 39710
63.278 34309
[2685204, 2756250, 4917150, 5635009, 13244685, 20112069, 20513864, 20962185]
9 8
[60.88897959183674, 62.5, 111.5, 127.77798185941043, 300.33299319727894, 456.0559863945578, 465.1669841269841, 475.33299319727894]
processing  3429 0.2
video seconds: 347
347 44100 15302700
[9.722, 127.889, 130.833, 158.389, 161.556, 162.778, 163.833, 165.167, 189.167, 190.0, 219.778, 222.778, 262.944, 284.722, 298.667, 299.167, 305.778, 307.556, 308.944, 320.5, 321.0, 329.5, 332.944, 341.778]
bg_len: 15302700, ding_len: 39710
190.0 36736
299.167 22050
321.0 22050
[428740, 5639904, 5769735, 6984954, 7124619, 7178509, 7225035, 7283864, 8342264, 9692209, 9824509, 11595830, 12556240, 13171214, 13484809, 13563219, 13624430, 14134050, 14530950, 14682830, 15072409]
24 21
[9.721995464852608, 127.88897959183673, 130.8329931972789, 158.3

processing  3442 0.002
video seconds: 349
349 44100 15390900
[0.056, 2.111, 3.556, 5.167, 7.556, 16.278, 19.0, 25.5, 27.278, 30.389, 32.389, 33.778, 37.389, 39.167, 40.0, 47.111, 48.556, 55.389, 56.389, 58.444, 66.389, 67.389, 69.556, 70.222, 72.889, 74.333, 76.333, 76.778, 78.222, 85.444, 86.111, 87.889, 88.611, 103.389, 106.722, 110.389, 131.056, 134.167, 138.889, 142.056, 151.722, 185.278, 186.389, 189.0, 192.722, 194.222, 200.944, 201.778, 206.944, 209.444, 218.722, 222.778, 226.944, 232.889, 238.667, 239.556, 242.778, 245.389, 246.611, 248.056, 257.611, 263.278, 264.889, 265.667, 267.722, 269.556, 270.333, 278.0, 278.833, 282.444, 286.611, 287.5, 288.333, 289.444, 296.167, 297.111, 298.389, 304.222, 305.278, 308.056, 310.722, 317.833, 322.889, 323.5, 324.0, 325.667, 326.944, 327.833, 328.5, 331.667, 335.611, 337.556, 338.0, 340.278, 342.333, 343.278]
bg_len: 15390900, ding_len: 39710
40.0 36736
70.222 29371
76.778 19624
86.111 29415
88.611 31841
201.778 36779
239.556 39205
265.667

processing  1354 0.02
video seconds: 285
285 44100 12568500
[0.056, 4.056, 6.333, 7.889, 9.278, 30.222, 40.556, 43.5, 46.0, 46.889, 49.333, 53.0, 53.722, 54.167, 55.0, 55.889, 57.0, 59.056, 59.722, 61.389, 65.667, 66.667, 76.556, 80.556, 86.611, 90.72222222222223, 120.889, 142.444, 148.611, 155.778, 156.778, 157.5, 158.389, 162.333, 162.889, 164.0, 168.556, 170.167, 179.778, 196.722, 199.333, 206.61111111111111, 222.222, 238.944, 242.833, 257.944, 260.278, 261.111, 263.444, 264.333, 265.111, 269.056, 269.556, 270.722, 274.167, 283.167]
bg_len: 12568500, ding_len: 39710
46.889 39204
53.722 31840
55.0 36736
59.722 29371
157.5 31841
162.889 24519
261.111 36736
264.333 39205
269.556 22050
[2469, 178869, 279285, 347904, 409159, 1332790, 1788519, 1918350, 2028600, 2175585, 2337300, 2388764, 2464704, 2513700, 2604369, 2707254, 2895914, 2940014, 3376119, 3552519, 3819545, 4000850, 5331204, 6281780, 6553745, 6869809, 6913909, 6984954, 7158885, 7232400, 7433319, 7504364, 7928209, 8675440, 879058

processing  2156 0.2
video seconds: 383
383 44100 16890300
[20.0, 38.222, 42.0, 44.056, 65.278, 69.66666666666667, 135.056, 166.056, 206.667, 208.0, 208.944, 251.0, 290.167, 302.77777777777777, 338.833, 347.833, 352.5, 376.833, 381.5, 382.0]
bg_len: 16890300, ding_len: 39710
382.0 22050
[882000, 1685590, 1852200, 1942869, 2878759, 3072300, 5955969, 7323069, 9114014, 9172800, 9214430, 11069100, 12796364, 13352500, 14942535, 15339435, 15545250, 16618335, 16824150]
20 19
[20.0, 38.22199546485261, 42.0, 44.05598639455782, 65.27798185941043, 69.66666666666667, 135.05598639455783, 166.05598639455783, 206.66698412698412, 208.0, 208.94399092970522, 251.0, 290.1669841269841, 302.77777777777777, 338.83299319727894, 347.83299319727894, 352.5, 376.83299319727894, 381.5]
processing  2195 0.8
video seconds: 354
354 44100 15611400
[39.5, 41.222, 76.889, 182.33333333333334, 218.5, 219.611, 223.278, 226.11111111111111, 249.444, 251.056, 258.444, 262.1111111111111, 272.111, 325.778, 328.222, 333.333, 33

processing  1447 0.007
video seconds: 318
318 44100 14023800
[2.556, 3.222, 4.833, 5.444, 6.056, 7.667, 8.5, 9.833, 11.278, 17.222, 23.111, 23.611, 28.111, 29.444, 31.556, 33.333, 35.667, 38.833, 40.0, 45.333, 49.889, 51.778, 52.444, 56.167, 58.444, 63.944, 64.778, 67.056, 69.222, 69.778, 73.778, 79.556, 80.611, 85.611, 87.611, 95.056, 101.0, 109.0, 122.889, 129.167, 129.833, 131.0, 131.611, 135.611, 137.0, 138.944, 141.111, 144.056, 148.444, 154.278, 157.611, 158.444, 160.167, 161.833, 163.556, 164.611, 167.444, 169.111, 171.278, 181.222, 184.611, 185.278, 194.611, 195.111, 198.444, 201.389, 209.056, 211.333, 218.167, 219.222, 226.778, 228.833, 234.222, 238.611, 239.111, 241.944, 243.389, 246.056, 247.611, 251.833, 254.889, 263.5, 273.222, 276.0, 276.722, 284.611, 289.278, 289.778, 291.0, 294.667, 295.944, 317.722]
bg_len: 14023800, ding_len: 39710
3.222 29371
5.444 26945
8.5 36736
23.611 22050
52.444 29371
64.778 36779
69.778 24519
129.833 29371
131.611 26945
158.444 36735
185.278 29

processing  3479 0.005
video seconds: 482
482 44100 21256200
[0.056, 4.0, 7.278, 8.222, 9.556, 13.389, 25.722, 26.944, 28.111, 28.722, 30.056, 34.222, 36.833, 38.056, 42.556, 47.833, 50.167, 53.778, 59.278, 62.556, 63.333, 70.056, 71.056, 76.556, 82.389, 84.167, 95.5, 96.222, 101.278, 102.111, 106.222, 108.611, 113.444, 114.389, 115.0, 116.833, 118.056, 123.889, 125.778, 130.944, 146.167, 152.167, 157.056, 158.056, 160.167, 160.667, 164.667, 165.722, 168.333, 169.5, 170.778, 171.778, 172.5, 176.556, 177.833, 178.5, 183.5, 184.5, 193.056, 199.389, 200.278, 203.111, 203.944, 205.722, 207.722, 210.111, 211.556, 213.056, 216.611, 219.611, 222.778, 223.611, 224.778, 226.0, 230.278, 231.222, 232.667, 234.056, 236.778, 238.333, 239.611, 241.556, 242.111, 245.667, 249.111, 250.778, 252.722, 257.556, 260.5, 260.55555555555554, 265.5, 268.944, 271.944, 273.278, 284.444, 288.111, 289.111, 295.111, 301.0, 310.389, 320.056, 323.778, 325.444, 327.056, 328.944, 335.167, 335.889, 341.0, 342.778, 343.6

In [11]:
704 * 44100

31046400